# Datasets & DataLoaders


理想情况下，我们希望我们处理数据集的代码与模型训练代码解耦，以获得更好的可读性和模块化性。
PyTorch提供了两个数据原语：``torch.utils.data.DataLoader``和``torch.utils.data.Dataset``，它们允许我们使用预加载的数据集以及自己的数据。
``Dataset``存储样本及其对应的标签，而``DataLoader``在``Dataset``周围包装了一个可迭代对象，以便轻松访问样本。


--------------




## 根据数据文件创建自己的Dataset

一个自定义的`Dataset`类必须实现三个函数：`__init__`、`__len__`和`__getitem__`。
接下来我们尝试使用SST-2数据集来实现自定义的Dataset类。SST-2是斯坦福提出的一个电影评论情感分类数据集，我们仅使用其中的dev集来进行实验。
让我们来看一下这个实现：dev.jsonl文件中一行存储了一条样例数据，其中"text","label"和"label_text"分别表示评论、情感标签值和情感标签。

接下来开始定义Dataset，其中file_path表示数据路径，split表示数据是训练集、验证集还是测试集。

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class SST2Dataset(Dataset):
    # `__init__`函数仅在实例化`Dataset`对象时运行一次。我们在这里加载所有数据。
    def __init__(self, file_path, split):
        file_path = f"{file_path}/{split}.jsonl"
        self.datas = pd.read_json(file_path, lines=True).to_dict(orient='records')

    # len() 函数返回数据集的样本数
    def __len__(self):
        return len(self.datas)

    # `__getitem__`函数从给定索引``idx``的数据集中加载并返回一个样本。
    def __getitem__(self, idx):
        return self.datas[idx]

--------------




## 使用DataLoaders准备数据进行训练
``Dataset``一次检索我们数据集的一个样本。在训练模型时，通常我们希望以"小批量"的方式传递样本，每个训练轮次重新打乱数据顺序以减少模型过拟合，并使用Python的``multiprocessing``来加速数据检索。

``DataLoader``是一个可迭代对象，它在简单的API中为我们抽象了这个复杂性。

### 文本数据批量化

我们知道自然语言是一些离散化的符号，如何将离散化的符号转换成计算机能理解的数字，再进行批量化（Tensor化）也是数据处理过程中重要的一步。

下面是一个基本的使用Bert Tokenizer进行文本批量化的演示代码。这里还不了解BERT和Tokenizer没关系，我们现在就当Tokenizer是一个文本转数字的工具即可。

In [2]:
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer

max_length = 128  # 最大文本长度

def my_collate_fn(batch):
    # 提取每个样本中的文本和标签信息
    texts = [sample['text'] for sample in batch]
    labels = [sample['label'] for sample in batch]

    # 截断或填充文本以确保不超过最大长度
    texts_list = []
    for text in texts:
        encoded_text = tokenizer.encode(text, truncation=True, max_length=max_length, padding='max_length')
        texts_list.append(encoded_text)

    texts_tensor = [torch.Tensor(text) for text in texts_list]

    # 将文本序列填充为相同长度
    texts_tensor = pad_sequence(texts_tensor, batch_first=True, padding_value=0)

    # 将标签转换为张量
    labels_tensor = torch.tensor(labels)

    return {'text': texts_tensor, 'label': labels_tensor}

接着我们分别定义训练集和测试集的DataLoader：

In [3]:
from torch.utils.data import DataLoader

training_data = SST2Dataset(".", "train")
test_data = SST2Dataset(".", "test")

# Load bert_tokenizer locally
tokenizer = AutoTokenizer.from_pretrained("./")

train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True, collate_fn=my_collate_fn)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True, collate_fn=my_collate_fn)

## 遍历DataLoader

我们已经将数据集加载到``DataLoader``中，并可以根据需要遍历数据集。
下面的每次迭代都返回一个批次的``text``和``label``（分别包含``batch_size=8``个文本输入和标签）。
因为我们指定了``shuffle=True``，所以在遍历完所有批次之后，数据将被重新打乱（如果需要更细粒度的控制数据加载顺序，请查看[Samplers](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler)）。

In [4]:

inputs = next(iter(train_dataloader))
train_texts, train_labels = inputs["text"], inputs["label"]
print(f"Texts batch shape: {train_texts.size()}")
print(f"Labels batch shape: {train_labels.size()}")
text= train_texts[0,:]
label = train_labels[0]
print(f"Text: {text}")
print(f"Label: {label}")

Texts batch shape: torch.Size([8, 128])
Labels batch shape: torch.Size([8])
Text: tensor([  101.,  1036.,  2054.,  1005.,  1055.,  1996.,  2845.,  2773.,  2005.,
        10166.,   999.,  1029.,  1005.,   102.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.

--------------




## Further Reading
- [torch.utils.data API](https://pytorch.org/docs/stable/data.html)

